In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("drive/My Drive/Task1_2inputs")
!python -c "import monai" || pip install -q "monai-weekly[nibabel, tqdm]"
import monai
from monai.data import  DataLoader, ImageDataset
from monai.transforms import (
    Resize, NormalizeIntensity, Activations, Compose, EnsureType, CenterSpatialCrop,
)

from monai.data import decollate_batch
from monai.networks.nets import DenseNet121
from skimage.morphology import disk, binary_dilation, binary_erosion, remove_small_objects
import pandas as pd
import numpy as np
import nibabel as nib
import torch
import scipy.ndimage as nd
import os
import matplotlib.pyplot as plt
import matplotlib
import nibabel.processing
from monai.utils import set_determinism

set_determinism(seed=123)
import warnings
warnings.filterwarnings('ignore')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
    data_dir = "data_testing"
    raw_data = pd.read_csv(os.path.join(data_dir, "Label.csv"))
    for _, c_row in raw_data.iterrows():
        patient = str(c_row['filename'].astype(int))
        pathCT1 = os.path.join(data_dir, "baseline", patient + ".nii.gz")
        pathCT1seg = os.path.join(data_dir, "mask", patient + "-label.nii.gz")
        # load mask and resample
        print("Processing : " + pathCT1)
        nii = nib.load(pathCT1seg)
        print("Original")
        print(nii.get_fdata().shape)
        matrix = (nii.header["pixdim"])[1:4]
        print(matrix)
        voxel_size = [1, 1, 1]
        fullImage1seg = np.round(nii.get_fdata()).astype(int)
        fullImage1seg[fullImage1seg != 1] = 0

        affine = np.eye(4)
        affine[:3, :3] = np.diag((nii.header["pixdim"])[1:4])
        pathtmp = os.path.join(data_dir, "tmp_brain_seg.nii.gz")
        empty_header = nib.Nifti1Header()
        clipped_img = nib.Nifti1Image(fullImage1seg, affine, empty_header)
        nib.save(clipped_img, pathtmp)
        nii = nib.load(pathtmp)
        nii = nibabel.processing.resample_to_output(nii, voxel_size)
        fullImage1seg = np.round(nii.get_fdata()).astype(int)
        fullImage1seg[fullImage1seg > 0] = 1

        # load brain window and resample
        window_center, window_width = 40, 80
        nii = nib.load(pathCT1)
        fullImage1 = nii.get_fdata()
        fullImage1[fullImage1 < 0] = 0
        fullImage1[fullImage1 > 200] = 0
        pathtmp = os.path.join(data_dir, "tmp_brain_seg.nii.gz")
        empty_header = nib.Nifti1Header()
        clipped_img = nib.Nifti1Image(fullImage1, affine, empty_header)
        nib.save(clipped_img, pathtmp)
        nii = nib.load(pathtmp)
        nii = nibabel.processing.resample_to_output(nii, voxel_size)
        fullImage1 = nii.get_fdata()
        img_min = window_center - window_width // 2
        img_max = window_center + window_width // 2
        fullImage1[fullImage1 < img_min] = img_min
        fullImage1[fullImage1 > img_max] = img_max
        fullImage1 = (fullImage1 - fullImage1.min()) / np.ptp(fullImage1)

        print("Resample (1,1,1)")
        print(fullImage1.shape)
        print(fullImage1seg.shape)
        # find the first slice which contains hematoma
        for slice1 in range(fullImage1seg.shape[2]):
            if fullImage1seg[:, :, slice1].sum() > 0:
                break
        # find the last slice which contains hematoma
        for slice2 in range(fullImage1seg.shape[2] - 1, 0, -1):
            if fullImage1seg[:, :, slice2].sum() > 0:
                break
        slice1 = slice1 - 16
        if slice1 < 0:
            slice1 = 0
        slice2 = slice1 + 64
        if slice2 > fullImage1seg.shape[2]:
            slice2 = fullImage1seg.shape[2]
        img1 = np.zeros([fullImage1.shape[0], fullImage1.shape[1], 64])
        img1seg = np.zeros([fullImage1seg.shape[0], fullImage1seg.shape[1], 64])
        img1[:, :, 0:slice2 - slice1] = fullImage1[:, :, slice1:slice2]
        img1seg[:, :, 0:slice2 - slice1] = fullImage1seg[:, :, slice1:slice2]

        img1 = 1.0 * (img1 - img1.min()) / np.ptp(img1)
        img1[img1 < 0.0001] = 0
        img_bw = img1.copy()
        img_bw[img_bw > 0] = 1
        for slice in range(0, img_bw.shape[2]):
            if img_bw[:, :, slice].sum() > 0:
                img_bw[:, :, slice] = binary_erosion(img_bw[:, :, slice].astype(np.uint8),
                                                     disk(4, dtype=bool))
                img_bw[:, :, slice] = remove_small_objects(img_bw[:, :, slice].astype(bool), 500)
                img_bw[:, :, slice] = binary_dilation(img_bw[:, :, slice].astype(np.uint8),
                                                      disk(4, dtype=bool))
                img_bw[:, :, slice] = nd.binary_fill_holes(img_bw[:, :, slice].astype(np.uint8))

        img1[img_bw == 0] = 0
        img1 = 1.0 * (img1 - img1.min()) / np.ptp(img1)

        image = np.zeros([2, img1.shape[0], img1.shape[1], img1.shape[2]])
        image[0, :, :, :] = img1
        image[1, :, :, :] = img1seg

        resizeImage = Resize([192, 192, 64])
        cropImage = CenterSpatialCrop(roi_size=(128, 128, 64))
        nomalizeImage = NormalizeIntensity()
        image = resizeImage(image)
        image = nomalizeImage(image)

        volume = np.array(image, dtype=np.float32)
        # Convert the numpy array into nifti file
        volume = nib.Nifti1Image(volume, np.eye(4))
        nib.save(volume, os.path.join(data_dir, "preprocessing", patient + "_brain_seg.nii.gz"))

Processing : data_testing/baseline/10028.nii.gz
Original
(512, 512, 32)
[0.46875 0.46875 5.     ]
Resample (1,1,1)
(241, 241, 156)
(241, 241, 156)
Processing : data_testing/baseline/10042.nii.gz
Original
(512, 512, 28)
[0.488281  0.488281  4.9998326]
Resample (1,1,1)
(251, 251, 136)
(251, 251, 136)
Processing : data_testing/baseline/10051.nii.gz
Original
(512, 527, 22)
[0.468     0.468     5.9917765]
Resample (1,1,1)
(241, 248, 127)
(241, 248, 127)
Processing : data_testing/baseline/10052.nii.gz
Original
(512, 512, 56)
[0.48828125 0.48828125 3.        ]
Resample (1,1,1)
(251, 251, 166)
(251, 251, 166)
Processing : data_testing/baseline/10056.nii.gz
Original
(512, 512, 36)
[0.488281 0.488281 5.      ]
Resample (1,1,1)
(251, 251, 176)
(251, 251, 176)


In [ ]:
    root_dir = "data_testing"
    data_dir = os.path.join(root_dir, "preprocessing")
    if 1:
        raw_data = pd.read_csv(os.path.join(root_dir, "Label.csv"))
        pathFilenames = []
        labels = []
        names = []
        for _, c_row in raw_data.iterrows():
            pathFilenames.append(os.path.join(data_dir, str(c_row['filename']) + "_brain_seg.nii.gz"))
            labels.append(c_row['label'])
            names.append(str(c_row['filename']))
        labels = np.asarray(labels).astype(int)
        pathFilenames = np.asarray(pathFilenames)
        names = np.asarray(names)
        if 1:
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
            model = monai.networks.nets.DenseNet121(
                spatial_dims=3, in_channels=2, out_channels=1, dropout_prob=0.1).to(device)
            model.cuda()
            model.load_state_dict(
                torch.load("1_best_metric_model_classification3d_1.pth"))
            model.eval()
            xval = pathFilenames
            yval = labels
            val_ds = ImageDataset(
                image_files=xval, labels=yval)
            val_loader = DataLoader(val_ds, batch_size=1, shuffle=False,
                                    num_workers=1, pin_memory=torch.cuda.is_available())
            y_pred_trans = Compose([EnsureType(), Activations(sigmoid=True)])
            y_trans = Compose([EnsureType()])
            with torch.no_grad():
                y_pred = torch.tensor([], dtype=torch.float32, device=device)
                y = torch.tensor([], dtype=torch.float32, device=device)
                for val_data in val_loader:
                    inputs, val_labels = val_data[0].to(
                        device), val_data[1].to(device)
                    val_outputs = model(inputs)
                    y_pred1 = val_outputs.flatten()
                    y1 = val_labels
                    y_pred = torch.cat([y_pred, y_pred1], dim=0)
                    y = torch.cat([y, y1], dim=0)
                y_onehot = [y_trans(i) for i in decollate_batch(y)]
                y_pred_act = [y_pred_trans(i) for i in decollate_batch(y_pred)]
                auc_metric1 = monai.metrics.ROCAUCMetric()
                auc_metric1(y_pred_act, y_onehot)
                print("AUC total= ")
                print(auc_metric1.aggregate())
                auc_metric1.reset()

AUC total= 
0.16666666666666666
